# Convert semantic segmentation dataset to instance segmentation dataset

This notebook demonstrates how a semantic segmentation dataset, with annotations
stored as PNG images, can be converted to a 3LC instance segmentation Table.

## Imports

In [ ]:
import json
from pathlib import Path

import cv2
import numpy as np
import tlc
from huggingface_hub import hf_hub_download

from tlc_tools.common import download_and_extract_zipfile

## Project Setup

In [ ]:
PROJECT_NAME = "3LC Tutorials"
DATASET_NAME = "ADE20k_toy_dataset"
TRANSIENT_DATA_PATH = "../../transient_data"

## Download the dataset

In [ ]:
DATASET_ROOT = (Path(TRANSIENT_DATA_PATH) / "ADE20k_toy_dataset").resolve()

if not DATASET_ROOT.exists():
    print("Downloading data...")
    download_and_extract_zipfile(
        url="https://www.dropbox.com/s/l1e45oht447053f/ADE20k_toy_dataset.zip?dl=1",
        location=TRANSIENT_DATA_PATH,
    )

## Fetch the label map from the Hugging Face Hub

In [ ]:
# load id2label mapping from a JSON on the hub
with open(
    hf_hub_download(
        repo_id="huggingface/label-files",
        filename="ade20k-id2label.json",
        repo_type="dataset",
    )
) as f:
    id2label = json.load(f)

categories = list(id2label.values())

In [ ]:
tlc.register_url_alias("ADE20K_TOY_DATASET_ROOT", DATASET_ROOT.as_posix())

## Load the images and segmentation maps





In [ ]:
image_paths = list(DATASET_ROOT.glob("**/images/training/*.jpg"))
segmentation_map_paths = list(DATASET_ROOT.glob("**/annotations/training/*.png"))

In [ ]:
# Call .to_relative() to ensure aliases are applied
image_paths = [tlc.Url(p).to_relative().to_str() for p in image_paths]
print(image_paths[0])

## Transform the segmentation maps to instance segmentation masks

In [ ]:
def single_channel_map_to_per_class_masks(map: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """Convert a single channel segmentation map to a stack of per-class masks.

    Args:
        map: A numpy array of shape (H, W) representing a single channel segmentation map.

    Returns:
        A tuple of two numpy arrays:
        - The first array is a stack of per-class masks of shape (H, W, N), where N is the number of classes in the map.
        - The second array is a numpy array of shape (N,) representing the class IDs present in the map.
    """
    masks = []
    labels = []
    for class_id in np.unique(map):
        mask = (map == class_id).astype(np.uint8)
        masks.append(mask)
        labels.append(class_id)
    return np.stack(masks, axis=-1), labels

In [ ]:
# Build the column of instance segmentations in the format required by 3LC
mask_dicts = []

for mask_path in segmentation_map_paths:
    map_np = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    h, w = map_np.shape
    masks, labels = single_channel_map_to_per_class_masks(map_np)

    mask_dicts.append(
        {
            "image_height": h,
            "image_width": w,
            "masks": masks,
            "instance_properties": {
                "label": labels,
            },
        },
    )

## Write the instance segmentation masks to a table

In [ ]:
table_writer = tlc.TableWriter(
    table_name="ADE20K-instance-segmentation",
    dataset_name=DATASET_NAME,
    project_name=PROJECT_NAME,
    column_schemas={
        "image": tlc.ImagePath("image"),
        "instances": tlc.InstanceSegmentationMasks(
            "instances",
            instance_properties_structure={
                "label": tlc.CategoricalLabel("label", categories),
            },
        ),
    },
    if_exists="rename",
)

In [ ]:
# Add all rows (images and instance segmentations) to the table in one go
table_writer.add_batch(
    {
        "image": image_paths,
        "instances": mask_dicts,
    }
)

In [ ]:
table = table_writer.finalize()

## Visualize a sample instance segmentation mask

In [ ]:
import matplotlib.pyplot as plt

example_mask = table[0]["instances"]["masks"][:, :, 0]

plt.imshow(example_mask, cmap="gray")
plt.axis("off")
plt.show()